# 📊 Задание по методам валидации в машинном обучении  
---  
> урок_№: 1    
домашняя_работа_№: 1  
исполнитель: Маъруф Ибрагимов  

## 1. Загрузка и предобработка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 📥 Загрузите набор данных Bank Customer Churn Prediction и обработайте пропуски

In [2]:
source_file='Bank Customer Churn Prediction.csv'

In [3]:
df=pd.read_csv(source_file)

In [4]:
df.isna().sum()

customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
dtype: int64

In [5]:
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


In [7]:
round(df.describe().T, 2)

,count,mean,std,min,25%,50%,75%,max
customer_id,10000.0,15690940.57,71936.19,15565701.00,15628528.25,15690738.00,15753233.75,15815690.00
credit_score,10000.0,650.53,96.65,350.00,584.00,652.00,718.00,850.00
age,10000.0,38.92,10.49,18.00,32.00,37.00,44.00,92.00
tenure,10000.0,5.01,2.89,0.00,3.00,5.00,7.00,10.00
balance,10000.0,76485.89,62397.41,0.00,0.00,97198.54,127644.24,250898.09
products_number,10000.0,1.53,0.58,1.00,1.00,1.00,2.00,4.00
credit_card,10000.0,0.71,0.46,0.00,0.00,1.00,1.00,1.00
active_member,10000.0,0.52,0.50,0.00,0.00,1.00,1.00,1.00
estimated_salary,10000.0,100090.24,57510.49,11.58,51002.11,100193.92,149388.25,199992.48
churn,10000.0,0.20,0.40,0.00,0.00,0.00,0.00,1.00


In [8]:
df.describe(include='object').T

,count,unique,top,freq
country,10000,3,France,5014
gender,10000,2,Male,5457


In [9]:
df['country'].unique().tolist()

['France', 'Spain', 'Germany']

> Пропусков в файле не обнаружено.

In [10]:
df2=df.drop(labels=['customer_id'], axis=1).copy()
df2.shape

(10000, 11)

### 🔄 Разделите данные на признаки (X) и целевую переменную (y)

In [11]:
target='churn'

X=df2.drop(labels=[target], axis=1)
y=df2[target]

print(X.shape)
print(y.shape)

(10000, 10)
(10000,)


### 📈 Выведите корреляционную сортированную таблицу признаков и таргета для определения самых важных признаков, которые влияют на целевую переменную (либо в виде сортированного графика)

In [12]:
corr=df2.select_dtypes(include='number').corr()
target_corr=(
    corr.query("index!=@target")[[target]]
    .assign(abs_corr_coef=lambda d_:d_[target].abs())
    .sort_values(by=['abs_corr_coef'], ascending=False)
)
target_corr

,churn,abs_corr_coef
age,0.285323,0.285323
active_member,-0.156128,0.156128
balance,0.118533,0.118533
products_number,-0.047820,0.047820
credit_score,-0.027094,0.027094
tenure,-0.014001,0.014001
estimated_salary,0.012097,0.012097
credit_card,-0.007138,0.007138


### 📊 Выведите распределение целевой переменной в процентном соотношении

In [13]:
y_props=y.value_counts(normalize=True)
y_props

churn
0    0.7963
1    0.2037
Name: proportion, dtype: float64

## 2. Удерживающая проверка (Hold-Out CV)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from catboost import CatBoostClassifier, Pool

### ✂️ Разделите данные на обучающую и тестовую выборки в соотношении 80/20.

In [15]:
seed=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=seed)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 10)
(2000, 10)
(8000,)
(2000,)


### 🧠 Обучите модель CatBoostClassifier с использованием обучающей выборки.

In [16]:
df2.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
{c:df2[c].nunique() for c in df2.columns}

{'credit_score': 460,
 'country': 3,
 'gender': 2,
 'age': 70,
 'tenure': 11,
 'balance': 6382,
 'products_number': 4,
 'credit_card': 2,
 'active_member': 2,
 'estimated_salary': 9999,
 'churn': 2}

In [18]:
df2.groupby('tenure')[target].value_counts(normalize=True).unstack().sort_index()

churn,0,1
tenure,,
0,0.769976,0.230024
1,0.775845,0.224155
2,0.808206,0.191794
3,0.788900,0.211100
4,0.794742,0.205258
5,0.793478,0.206522
6,0.797311,0.202689
7,0.827821,0.172179
8,0.807805,0.192195


In [19]:
cat_features=['country', 'gender', 'products_number', 'credit_card', 'active_member', 'tenure']

In [20]:
train_pool=Pool(data=X_train, label=y_train, cat_features=cat_features)
test_pool=Pool(data=X_test, label=y_test, cat_features=cat_features)

In [21]:
model=CatBoostClassifier(
    random_state=seed,
    depth=2,
    eval_metric='AUC',
    verbose=0
)

In [22]:
model.fit(train_pool, eval_set=test_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### 📊 Оцените её производительность на тестовой выборке, используя метрику AUC.

In [23]:
train_recall=metrics.recall_score(y_train, model.predict(X_train))
train_precision=metrics.precision_score(y_train, model.predict(X_train))
train_accuracy=metrics.accuracy_score(y_train, model.predict(X_train))
train_auc=metrics.roc_auc_score(y_train, model.predict(X_train))

test_recall=metrics.recall_score(y_test, model.predict(X_test))
test_precision=metrics.precision_score(y_test, model.predict(X_test))
test_accuracy=metrics.accuracy_score(y_test, model.predict(X_test))
test_auc=metrics.roc_auc_score(y_test, model.predict(X_test))

In [24]:
print(f'recall: train - {round(train_recall, 2)}, test - {round(test_recall, 2)}')
print(f'roc_auc: train - {round(train_auc, 2)}, test - {round(test_auc, 2)}')

recall: train - 0.51, test - 0.51
roc_auc: train - 0.74, test - 0.73


In [25]:
metrics_labels=['recall', 'precision', 'accuracy', 'roc_auc']
cv_comparison=pd.DataFrame({
        'cv_type':['Hold-Out CV']*8, 
        'metric':metrics_labels*2, 
        'stat':['mean']*4+['std']*4,
        'value':[test_recall, test_precision, test_accuracy, test_auc]+[0]*4
})

In [26]:
cv_comparison

,cv_type,metric,stat,value
0,Hold-Out CV,recall,mean,0.513995
1,Hold-Out CV,precision,mean,0.737226
2,Hold-Out CV,accuracy,mean,0.868500
3,Hold-Out CV,roc_auc,mean,0.734595
4,Hold-Out CV,recall,std,0.000000
5,Hold-Out CV,precision,std,0.000000
6,Hold-Out CV,accuracy,std,0.000000
7,Hold-Out CV,roc_auc,std,0.000000


## 3. k-блочная перекрестная проверка (k-Fold CV)

In [27]:
from sklearn.model_selection import cross_val_score

### 🔁 Реализуйте k-Fold перекрестную проверку с использованием 5 блоков.

In [28]:
model_2=CatBoostClassifier(
    random_state=seed,
    depth=2,
    iterations=100,
    eval_metric='AUC',
    verbose=0,
    cat_features=cat_features
)

### 📊 Вычислите среднее значение AUC для всех блоков.

In [29]:
def compute_n_record(metric, cv_type, cv=10):
    scores = cross_val_score(model_2, X,y, cv=cv, scoring=metric)
    cv_comparison.loc[len(cv_comparison)]=[cv_type, metric, 'mean', scores.mean()]
    cv_comparison.loc[len(cv_comparison)]=[cv_type, metric, 'std', scores.std()]

In [30]:
for label in metrics_labels:
    compute_n_record(metric=label, cv_type='k-Fold CV')

In [31]:
cv_comparison.query("cv_type=='k-Fold CV'")

,cv_type,metric,stat,value
8,k-Fold CV,recall,mean,0.464906
9,k-Fold CV,recall,std,0.042756
10,k-Fold CV,precision,mean,0.767766
11,k-Fold CV,precision,std,0.025687
12,k-Fold CV,accuracy,mean,0.862300
13,k-Fold CV,accuracy,std,0.008026
14,k-Fold CV,roc_auc,mean,0.861096
15,k-Fold CV,roc_auc,std,0.009622


## 4. Стратифицированный k-Fold CV

In [32]:
from sklearn.model_selection import StratifiedKFold

### ⚖️ Выполните стратифицированную k-Fold перекрестную проверку

In [33]:
skf=StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
for label in metrics_labels:
    compute_n_record(metric=label, cv_type='Stratified k-Fold CV', cv=skf)

### 🔍 Сравните результаты со стандартной k-Fold проверкой

In [34]:
round((
    cv_comparison
    .query("cv_type.isin(['k-Fold CV', 'Stratified k-Fold CV'])")
    .pivot_table(
        index=['metric'],
        columns=['cv_type', 'stat'],
        values='value',
        aggfunc='mean'
    )
),4)

cv_type   Stratified k-Fold CV         k-Fold CV        
stat                      mean     std      mean     std
metric                                                  
accuracy                0.8599  0.0075    0.8623  0.0080
precision               0.7601  0.0282    0.7678  0.0257
recall                  0.4571  0.0372    0.4649  0.0428
roc_auc                 0.8611  0.0172    0.8611  0.0096

### 📝 Объясните, в чем преимущество стратифицированного подхода

>Преимущество стратифицированной кросс-валидации в том, что в каждом фолде, она стремится делить датасет на тренировочный и тестовый так, чтобы в каждом из сетов сохранялись такие же пропорции классов как и в самом датасете.

## 5. Перекрестная проверка с исключением одного (LOOCV)

In [35]:
from sklearn.model_selection import LeaveOneOut

### 🔄 Реализуйте метод LOOCV

In [36]:
loo=LeaveOneOut()

In [37]:
df_sample=df2.groupby(target).sample(frac=.1, random_state=seed)

In [38]:
df_sample

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
8165,606,Spain,Male,37,8,154712.58,2,1,0,89099.18,0
6409,683,France,Female,33,9,0.00,2,1,1,38784.42,0
2157,744,France,Female,44,3,0.00,2,1,1,189016.14,0
1618,697,France,Male,27,8,141223.68,2,1,0,90591.15,0
1633,562,France,Male,35,7,0.00,1,0,0,48869.67,0
...,...,...,...,...,...,...,...,...,...,...,...
5187,683,France,Female,30,2,0.00,2,0,1,100496.84,1
2038,621,France,Male,47,0,0.00,1,1,1,133831.37,1
335,485,France,Male,29,7,182123.79,1,1,0,116828.51,1
9440,593,Spain,Male,27,10,0.00,3,0,0,94620.00,1


In [39]:
df_sample['churn'].value_counts(normalize=True)

churn
0    0.796
1    0.204
Name: proportion, dtype: float64

In [40]:
X_sample=df_sample.drop(labels=[target], axis=1)
y_sample=df_sample[target]

In [41]:
y_true=[]
y_pred=[]

for train_ix, test_ix in loo.split(X_sample):
    X_sample_train, X_sample_test = X_sample.iloc[train_ix], X_sample.iloc[test_ix]
    y_sample_train, y_sample_test = y_sample.iloc[train_ix], y_sample.iloc[test_ix]

    model_2.fit(X_sample_train, y_sample_train)

    prediction=model_2.predict(X_sample_test)

    y_pred.append(prediction[0])
    y_true.append(y_sample_test.iloc[0])


In [42]:
loo_cv=pd.DataFrame({
        'cv_type':['LOO CV']*8, 
        'metric':metrics_labels*2, 
        'stat':['mean']*4+['std']*4,
        'value':[
            metrics.recall_score(y_true, y_pred), 
            metrics.precision_score(y_true, y_pred), 
            metrics.accuracy_score(y_true, y_pred), 
            metrics.roc_auc_score(y_true, y_pred)
        ]+[0]*4
})
cv_comparison=pd.concat([cv_comparison, loo_cv], ignore_index=True)
    

## 6. Подготовка сравнительной таблицы

In [43]:
round(
    cv_comparison.pivot_table(
        index=['metric', 'stat'],
        columns=['cv_type'],
        values='value',
        aggfunc='mean'
    )[['Hold-Out CV', 'k-Fold CV', 'Stratified k-Fold CV', 'LOO CV']],
    4
)

cv_type         Hold-Out CV  k-Fold CV  Stratified k-Fold CV  LOO CV
metric    stat                                                      
accuracy  mean       0.8685     0.8623                0.8599  0.8520
          std        0.0000     0.0080                0.0075  0.0000
precision mean       0.7372     0.7678                0.7601  0.7917
          std        0.0000     0.0257                0.0282  0.0000
recall    mean       0.5140     0.4649                0.4571  0.3725
          std        0.0000     0.0428                0.0372  0.0000
roc_auc   mean       0.7346     0.8611                0.8611  0.6737
          std        0.0000     0.0096                0.0172  0.0000

## 7. Задание на доп балл

### 🏆 Выведите таблицу с метриками для порогов отсечения 0.05, 0.1, 0.15, 0.25, 0.3 для положительного класса (Только для Hold-Out CV)

In [44]:
thresholds=[.05, .1, .15, .25, .3]
thresh_results=[]

for t in thresholds:
    train_dict={}
    test_dict={}

    y_train_pred=(model.predict_proba(X_train)[:, 1]>t).astype(int)
    y_test_pred=(model.predict_proba(X_test)[:, 1]>t).astype(int)

    train_dict['threshold']=t
    train_dict['set']='train'
    train_dict['recall']=metrics.recall_score(y_train, y_train_pred)
    train_dict['precision']=metrics.precision_score(y_train, y_train_pred)
    train_dict['accuracy']=metrics.accuracy_score(y_train, y_train_pred)
    train_dict['roc_auc']=metrics.roc_auc_score(y_train, y_train_pred)

    test_dict['threshold']=t
    test_dict['set']='test'
    test_dict['recall']=metrics.recall_score(y_test, y_test_pred)
    test_dict['precision']=metrics.precision_score(y_test, y_test_pred)
    test_dict['accuracy']=metrics.accuracy_score(y_test, y_test_pred)
    test_dict['roc_auc']=metrics.roc_auc_score(y_test, y_test_pred)

    thresh_results.extend([train_dict, test_dict])

In [45]:
t_df=pd.DataFrame.from_records(thresh_results)

In [46]:
round(t_df.set_index(['threshold', 'set']), 4)

recall  precision  accuracy  roc_auc
threshold set                                        
0.05      train  0.9738     0.2831    0.4878   0.6679
          test   0.9618     0.2666    0.4725   0.6573
0.10      train  0.9173     0.3635    0.6529   0.7509
          test   0.9109     0.3459    0.6440   0.7448
0.15      train  0.8552     0.4371    0.7439   0.7852
          test   0.8524     0.4151    0.7350   0.7794
0.25      train  0.7275     0.5584    0.8258   0.7893
          test   0.7735     0.5640    0.8380   0.8137
0.30      train  0.6746     0.6230    0.8492   0.7845
          test   0.7150     0.6082    0.8535   0.8012